In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cicids2017/combined_cicids2017.csv
/kaggle/input/cicids2017/df_cleaned_3.csv
/kaggle/input/cicids2017/df_cleaned_4.csv
/kaggle/input/unsw-nb15-mlp-model-h5/unsw_nb15_mlp_model.h5
/kaggle/input/checkpoint-model/checkpoint_model-7.keras
/kaggle/input/model-saved/checkpoint_model_gru-3.keras
/kaggle/input/model-saved/checkpoint_model_gru-4.keras
/kaggle/input/model-saved/checkpoint_model-7.keras
/kaggle/input/model-saved/checkpoint_model_lstm.keras
/kaggle/input/model-saved/checkpoint_model_gru-2.keras
/kaggle/input/model-saved/checkpoint_model_lstm-4.keras
/kaggle/input/model-saved/checkpoint_model_gru-5.keras
/kaggle/input/model-saved/checkpoint_model_gru-6.keras
/kaggle/input/model-saved/cicids2017_fine_tuned_model.h5
/kaggle/input/model-saved/checkpoint_model_lstm-2.keras
/kaggle/input/weights/checkpoint.weights-4.h5
/kaggle/input/weights/checkpoint.weights-3.h5
/kaggle/input/weights/checkpoint.weights-2.h5


In [2]:
import pandas as pd
df_cleaned_3 = pd.read_csv('/kaggle/input/cicids2017/df_cleaned_3.csv')
df_cleaned_4 = pd.read_csv('/kaggle/input/cicids2017/df_cleaned_4.csv')

print(df_cleaned_3.shape)
print(df_cleaned_4.shape)

(2520798, 59)
(2520798, 56)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam


fine_tuned_model = load_model('/kaggle/input/model-saved/checkpoint_model-7.keras')
fine_tuned_model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 56)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 20)                  │           1,140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15)                  │             315 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,507 (9.80 KB)

 Trainable params: 315 (1.23 KB)

 Non-trainable params: 1,560 (6.09 KB)

 Optimizer params: 632 (2.47 KB)

In [4]:
# fine_tuned_model.compile(
#     optimizer=Adam(learning_rate=1e-4),
#     loss="sparse_categorical_crossentropy",
#     metrics=["accuracy"]
# )

In [5]:
# Separate features (X) and labels (y)
X = df_cleaned_4.values  # Features
y = df_cleaned_3['Encoded_Label'].values  # Encoded labels



In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

scaler = StandardScaler()
smote = SMOTE(random_state=42)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

# Path to save the model
checkpoint_filepath = '/kaggle/working/checkpoint_model.keras'
# checkpoint_filepath_weight = '/kaggle/working/checkpoint_model.weights.h5'
#checkpoint_filepath_f1 = '/kaggle/working/checkpoint_model_f1.keras'
#checkpoint_filepath_roc_auc = '/kaggle/working/checkpoint_model_roc_auc.keras'

# Configure ModelCheckpoint to save the full model
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,  # Set to False to save the full model
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,  # Only save when 'val_accuracy' improves
    verbose=1
)

# model_checkpoint_weight = ModelCheckpoint(
#     filepath=checkpoint_filepath_weight,
#     save_weights_only=True,  # Set to False to save the full model
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True,  # Only save when 'val_accuracy' improves
#     verbose=1
# )

# class PerformanceMetrics(Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         y_pred = self.model.predict(self.validation_data[0])
#         y_true = self.validation_data[1]

#         # Assuming binary classification with a threshold of 0.5
#         y_pred = (y_pred > 0.5).astype(int)
        
#         # Calculate F1 Score for the minority class
#         f1 = f1_score(y_true, y_pred, average='binary')
#         logs['val_f1_score'] = f1

#         # Calculate ROC-AUC Score
#         roc_auc = roc_auc_score(y_true, y_pred)
#         logs['val_roc_auc'] = roc_auc

#         print(f"Epoch {epoch+1}: val_f1_score = {f1:.4f}, val_roc_auc = {roc_auc:.4f}")

# # Add this callback to your model training
# metrics_callback = PerformanceMetrics()

# # Checkpoint to save the best model based on the highest validation F1 score
# model_checkpoint_f1 = ModelCheckpoint(
#     filepath= checkpoint_filepath_f1,
#     save_weights_only=False,  # Set to False to save the full model
#     monitor='val_f1_score',
#     mode='max',
#     save_best_only=True,
#     verbose=1
# )

# # Checkpoint to save the best model based on the highest validation ROC-AUC score
# model_checkpoint_roc_auc = ModelCheckpoint(
#     filepath= checkpoint_filepath_roc_auc,
#     save_weights_only=False,  # Set to False to save the full model
#     monitor='val_roc_auc',
#     mode='max',
#     save_best_only=True,
#     verbose=1
# )

In [8]:
# Initial split into training and test sets
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    X, y, test_size=0.3, random_state=44, stratify=y
)

# Further split the training set into training and validation sets
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(
    X_train_2, y_train_2, test_size=0.2, random_state=44, stratify=y_train_2
)

# Check for extremely large values
print("Max value in X_train_2:", np.max(X_train_2))
print("Min value in X_train_2:", np.min(X_train_2))

X_train_2_scaled = scaler.fit_transform(X_train_2)
# Get the original class distribution
class_counts_2 = Counter(y_train_2)
print("Original class distribution:", class_counts_2)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_2 = class_counts_2[min(class_counts_2, key=class_counts_2.get)]
desired_majority_size_2 = minority_class_size_2 * 5

# Create the sampling strategy dictionary
sampling_strategy_2 = {0: desired_majority_size_2, 1: minority_class_size_2}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_2 = RandomUnderSampler(sampling_strategy=sampling_strategy_2, random_state=42)
X_resampled_2, y_resampled_2 = undersampler_2.fit_resample(X_train_2, y_train_2)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_2))

# Apply SMOTE on the smaller subset
X_train_resampled_2, y_train_resampled_2 = smote.fit_resample(X_resampled_2, y_resampled_2)

#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_2))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_2))

Max value in X_train_2: 2071000000.0
Min value in X_train_2: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [9]:
# Evaluate the model on the test set
test_loss, test_accuracy = fine_tuned_model.evaluate(X_test_2, y_test_2, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

23633/23633 - 27s - 1ms/step - accuracy: 0.7024 - loss: 94021.3750
Test Loss: 94021.375
Test Accuracy: 0.7023894786834717


In [10]:
from sklearn.metrics import classification_report, f1_score

# Get predictions on the test set
y_pred_probs = fine_tuned_model.predict(X_test_2)  # Get probability scores
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert to binary labels (for binary classification)

# Compute F1 Score
f1 = f1_score(y_test_2, y_pred)

# Print the classification report (includes Precision, Recall, and F1-score)
print(classification_report(y_test_2, y_pred))

# Print F1 Score separately
print("F1 Score:", f1)

23633/23633 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step


ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [ ]:
import numpy as np

# Convert one-hot encoded y_test_2 to categorical labels
if len(y_test_2.shape) > 1 and y_test_2.shape[1] > 1:
    y_test_2 = np.argmax(y_test_2, axis=1)

# Convert predicted probabilities to categorical labels
if len(y_pred_probs.shape) > 1 and y_pred_probs.shape[1] > 1:
    y_pred = np.argmax(y_pred_probs, axis=1)
else:
    y_pred = (y_pred_probs > 0.5).astype(int)  # For binary classification


In [11]:
print("y_test_2 shape:", y_test_2.shape)
print("y_pred shape:", y_pred.shape)

y_test_2 shape: (756240,)
y_pred shape: (756240, 15)


In [12]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the index of the max probability

# Compute F1 Score
f1 = f1_score(y_test_2, y_pred_labels, average="weighted")  # "weighted" handles class imbalance
print("F1 Score:", f1)

# Print classification report
print(classification_report(y_test_2, y_pred_labels))


F1 Score: 0.7728965656498958
              precision    recall  f1-score   support

           0       0.98      0.68      0.80    628518
           1       0.02      0.60      0.05       584
           2       0.51      0.73      0.60     38404
           3       0.13      0.93      0.22      3086
           4       0.79      0.84      0.82     51854
           5       0.06      0.80      0.11      1568
           6       0.04      0.52      0.08      1616
           7       0.12      0.94      0.22      1779
           8       0.09      1.00      0.16         3
           9       0.00      0.55      0.00        11
          10       0.37      0.95      0.54     27208
          11       0.17      0.89      0.29       966
          12       0.04      0.89      0.07       441
          13       0.00      1.00      0.01         6
          14       0.00      0.01      0.00       196

    accuracy                           0.70    756240
   macro avg       0.22      0.75      0.26    7562

In [18]:
# Initial split into training and test sets
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(
    X, y, test_size=0.3, random_state=45, stratify=y
)

# Further split the training set into training and validation sets
X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(
    X_train_3, y_train_3, test_size=0.3, random_state=44, stratify=y_train_3
)

# Check for extremely large values
print("Max value in X_train_3:", np.max(X_train_3))
print("Min value in X_train_3:", np.min(X_train_3))

X_train_3_scaled = scaler.fit_transform(X_train_3)

# Get the original class distribution
class_counts_3 = Counter(y_train_3)
print("Original class distribution:", class_counts_3)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_3 = class_counts_3[min(class_counts_3, key=class_counts_3.get)]
desired_majority_size_3 = minority_class_size_3 * 5

# Create the sampling strategy dictionary
sampling_strategy_3 = {0: desired_majority_size_3, 1: minority_class_size_3}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_3 = RandomUnderSampler(sampling_strategy=sampling_strategy_3, random_state=42)
X_resampled_3, y_resampled_3 = undersampler_3.fit_resample(X_train_3, y_train_3)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_3))

# Apply SMOTE on the smaller subset
X_train_resampled_3, y_train_resampled_3 = smote.fit_resample(X_resampled_3, y_resampled_3)

#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_3))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_3))



Max value in X_train_3: 2071000000.0
Min value in X_train_3: -9663668122.0
Original class distribution: Counter({0: 1026577, 4: 84694, 2: 62727, 10: 44440, 3: 5040, 7: 2906, 6: 2638, 5: 2562, 11: 1577, 1: 955, 12: 720, 14: 319, 9: 18, 13: 11, 8: 6})
Class distribution after undersampling: Counter({4: 84694, 2: 62727, 10: 44440, 3: 5040, 7: 2906, 6: 2638, 5: 2562, 11: 1577, 12: 720, 14: 319, 0: 30, 9: 18, 13: 11, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 84694, 2: 62727, 10: 44440, 3: 5040, 7: 2906, 6: 2638, 5: 2562, 11: 1577, 12: 720, 14: 319, 0: 30, 9: 18, 13: 11, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 84694, 1: 84694, 2: 84694, 3: 84694, 4: 84694, 5: 84694, 6: 84694, 7: 84694, 8: 84694, 9: 84694, 10: 84694, 11: 84694, 12: 84694, 13: 84694, 14: 84694})


In [30]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_3,  # Features from CICIDS2017
    y_train_resampled_3,  # Labels from CICIDS2017
    validation_data=(X_val_3, y_val_3),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.41403

Epoch 1: val_accuracy did not improve from 0.41403
636/636 - 3s - 5ms/step - accuracy: 0.7930 - loss: 2659.8577 - val_accuracy: 0.3899 - val_loss: 2818298.2500
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.41403

Epoch 2: val_accuracy did not improve from 0.41403
636/636 - 3s - 5ms/step - accuracy: 0.7936 - loss: 2697.4426 - val_accuracy: 0.3671 - val_loss: 2857787.7500
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.41403

Epoch 3: val_accuracy did not improve from 0.41403
636/636 - 3s - 5ms/step - accuracy: 0.7934 - loss: 2663.2407 - val_accuracy: 0.3541 - val_loss: 2873420.5000
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.41403

Epoch 4: val_accuracy did not improve from 0.41403
636/636 - 3s - 5ms/step - accuracy: 0.7936 - loss: 2650.6018 - val_accuracy: 0.4040 - val_loss: 2878119.7500
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.41403

Epoch 5: val_accuracy did not impro

In [33]:

fine_tuned_model.load_weights('/kaggle/working/checkpoint_model.weights.h5')

# Now evaluate the model on the test set
test_loss, test_accuracy = fine_tuned_model.evaluate(X_test_3, y_test_3, verbose=2)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

23633/23633 - 25s - 1ms/step - accuracy: 0.4452 - loss: 4842540.0000
Test Loss: 4842540.0
Test Accuracy: 0.4451615810394287


In [34]:
# Initial split into training and test sets
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(
    X, y, test_size=0.3, random_state=46, stratify=y
)

# Further split the training set into training and validation sets
X_train_4, X_val_4, y_train_4, y_val_4 = train_test_split(
    X_train_4, y_train_4, test_size=0.2, random_state=46, stratify=y_train_4
)

# Check for extremely large values
print("Max value in X_train_4:", np.max(X_train_4))
print("Min value in X_train_4:", np.min(X_train_4))

X_train_4_scaled = scaler.fit_transform(X_train_4)

# Get the original class distribution
class_counts_4 = Counter(y_train_4)
print("Original class distribution:", class_counts_4)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_4 = class_counts_4[min(class_counts_4, key=class_counts_4.get)]
desired_majority_size_4 = minority_class_size_4 * 5

# Create the sampling strategy dictionary
sampling_strategy_4 = {0: desired_majority_size_4, 1: minority_class_size_4}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_4 = RandomUnderSampler(sampling_strategy=sampling_strategy_4, random_state=42)
X_resampled_4, y_resampled_4 = undersampler_4.fit_resample(X_train_4, y_train_4)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_4))

# Apply SMOTE on the smaller subset
X_train_resampled_4, y_train_resampled_4 = smote.fit_resample(X_resampled_4, y_resampled_4)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_4))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_4))



Max value in X_train_4: 2071000000.0
Min value in X_train_4: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [35]:
# Initial split into training and test sets
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(
    X, y, test_size=0.3, random_state=47, stratify=y
)

# Further split the training set into training and validation sets
X_train_5, X_val_5, y_train_5, y_val_5 = train_test_split(
    X_train_5, y_train_5, test_size=0.2, random_state=47, stratify=y_train_5
)

# Check for extremely large values
print("Max value in X_train_5:", np.max(X_train_5))
print("Min value in X_train_5:", np.min(X_train_5))

X_train_5_scaled = scaler.fit_transform(X_train_5)

# Get the original class distribution
class_counts_5 = Counter(y_train_5)
print("Original class distribution:", class_counts_5)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_5 = class_counts_5[min(class_counts_5, key=class_counts_5.get)]
desired_majority_size_5 = minority_class_size_5 * 5

# Create the sampling strategy dictionary
sampling_strategy_5 = {0: desired_majority_size_5, 1: minority_class_size_5}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_5 = RandomUnderSampler(sampling_strategy=sampling_strategy_5, random_state=42)
X_resampled_5, y_resampled_5 = undersampler_5.fit_resample(X_train_5, y_train_5)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_5))

# Apply SMOTE on the smaller subset
X_train_resampled_5, y_train_resampled_5 = smote.fit_resample(X_resampled_5, y_resampled_5)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_5))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_5))



Max value in X_train_5: 2071000000.0
Min value in X_train_5: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [36]:
# Initial split into training and test sets
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(
    X, y, test_size=0.3, random_state=48, stratify=y
)

# Further split the training set into training and validation sets
X_train_6, X_val_6, y_train_6, y_val_6 = train_test_split(
    X_train_6, y_train_6, test_size=0.2, random_state=48, stratify=y_train_6
)

# Check for extremely large values
print("Max value in X_train_6:", np.max(X_train_6))
print("Min value in X_train_6:", np.min(X_train_6))

X_train_6_scaled = scaler.fit_transform(X_train_6)

# Get the original class distribution
class_counts_6 = Counter(y_train_6)
print("Original class distribution:", class_counts_6)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_6 = class_counts_6[min(class_counts_6, key=class_counts_6.get)]
desired_majority_size_6 = minority_class_size_6 * 5

# Create the sampling strategy dictionary
sampling_strategy_6 = {0: desired_majority_size_6, 1: minority_class_size_6}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_6 = RandomUnderSampler(sampling_strategy=sampling_strategy_6, random_state=42)
X_resampled_6, y_resampled_6 = undersampler_6.fit_resample(X_train_6, y_train_6)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_6))

# Apply SMOTE on the smaller subset
X_train_resampled_6, y_train_resampled_6 = smote.fit_resample(X_resampled_6, y_resampled_6)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_6))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_6))




Max value in X_train_6: 2071000000.0
Min value in X_train_6: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [37]:

# Initial split into training and test sets
X_train_7, X_test_7, y_train_7, y_test_7 = train_test_split(
    X, y, test_size=0.3, random_state=49, stratify=y
)

# Further split the training set into training and validation sets
X_train_7, X_val_7, y_train_7, y_val_7 = train_test_split(
    X_train_7, y_train_7, test_size=0.2, random_state=49, stratify=y_train_7
)

# Check for extremely large values
print("Max value in X_train_7:", np.max(X_train_7))
print("Min value in X_train_7:", np.min(X_train_7))

scaler = StandardScaler()
X_train_7_scaled = scaler.fit_transform(X_train_7)

# Get the original class distribution
class_counts_7 = Counter(y_train_7)
print("Original class distribution:", class_counts_7)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_7 = class_counts_7[min(class_counts_7, key=class_counts_7.get)]
desired_majority_size_7 = minority_class_size_7 * 5

# Create the sampling strategy dictionary
sampling_strategy_7 = {0: desired_majority_size_7, 1: minority_class_size_7}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_7 = RandomUnderSampler(sampling_strategy=sampling_strategy_7, random_state=42)
X_resampled_7, y_resampled_7 = undersampler_7.fit_resample(X_train_7, y_train_7)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_7))

# Apply SMOTE on the smaller subset
smote = SMOTE(random_state=42)
X_train_resampled_7, y_train_resampled_7 = smote.fit_resample(X_resampled_7, y_resampled_7)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_7))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_7))




Max value in X_train_7: 2071000000.0
Min value in X_train_7: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [38]:
# Initial split into training and test sets
X_train_8, X_test_8, y_train_8, y_test_8 = train_test_split(
    X, y, test_size=0.3, random_state=50, stratify=y
)

# Further split the training set into training and validation sets
X_train_8, X_val_8, y_train_8, y_val_8 = train_test_split(
    X_train_8, y_train_8, test_size=0.2, random_state=50, stratify=y_train_8
)

# Check for extremely large values
print("Max value in X_train_8:", np.max(X_train_8))
print("Min value in X_train_8:", np.min(X_train_8))

X_train_8_scaled = scaler.fit_transform(X_train_8)

# Get the original class distribution
class_counts_8 = Counter(y_train_8)
print("Original class distribution:", class_counts_8)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_8 = class_counts_8[min(class_counts_8, key=class_counts_8.get)]
desired_majority_size_8 = minority_class_size_8 * 5

# Create the sampling strategy dictionary
sampling_strategy_8 = {0: desired_majority_size_8, 1: minority_class_size_8}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_8 = RandomUnderSampler(sampling_strategy=sampling_strategy_8, random_state=42)
X_resampled_8, y_resampled_8 = undersampler_8.fit_resample(X_train_8, y_train_8)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_8))

# Apply SMOTE on the smaller subset
X_train_resampled_8, y_train_resampled_8 = smote.fit_resample(X_resampled_8, y_resampled_8)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_8))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_8))




Max value in X_train_8: 2071000000.0
Min value in X_train_8: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [39]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_4,  # Features from CICIDS2017
    y_train_resampled_4,  # Labels from CICIDS2017
    validation_data=(X_val_4, y_val_4),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.44629

Epoch 1: val_accuracy did not improve from 0.44629
726/726 - 4s - 5ms/step - accuracy: 0.6985 - loss: 163774.4375 - val_accuracy: 0.3160 - val_loss: 4247741.0000
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.44629

Epoch 2: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7023 - loss: 146473.1875 - val_accuracy: 0.3112 - val_loss: 4028693.2500
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.44629

Epoch 3: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7223 - loss: 135300.8125 - val_accuracy: 0.3039 - val_loss: 3874640.0000
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.44629

Epoch 4: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7194 - loss: 126370.4922 - val_accuracy: 0.3206 - val_loss: 3769123.2500
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.44629

Epoch 5: val_accuracy did n

In [40]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_5,  # Features from CICIDS2017
    y_train_resampled_5,  # Labels from CICIDS2017
    validation_data=(X_val_5, y_val_5),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.44629

Epoch 1: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7382 - loss: 5821.7168 - val_accuracy: 0.3660 - val_loss: 137239.3125
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.44629

Epoch 2: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7575 - loss: 4359.3535 - val_accuracy: 0.3543 - val_loss: 143392.8906
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.44629

Epoch 3: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7611 - loss: 3990.4233 - val_accuracy: 0.3479 - val_loss: 165200.2344
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.44629

Epoch 4: val_accuracy did not improve from 0.44629
726/726 - 3s - 4ms/step - accuracy: 0.7639 - loss: 3606.1958 - val_accuracy: 0.2764 - val_loss: 192569.0625
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.44629

Epoch 5: val_accuracy did not improve f

In [41]:

# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_6,  # Features from CICIDS2017
    y_train_resampled_6,  # Labels from CICIDS2017
    validation_data=(X_val_6, y_val_6),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy improved from 0.44629 to 0.46319, saving model to /kaggle/working/checkpoint_model.keras

Epoch 1: val_accuracy improved from 0.44629 to 0.46319, saving model to /kaggle/working/checkpoint_model.weights.h5
726/726 - 3s - 4ms/step - accuracy: 0.7434 - loss: 11598.0029 - val_accuracy: 0.4632 - val_loss: 419646.3750
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.46319

Epoch 2: val_accuracy did not improve from 0.46319
726/726 - 3s - 4ms/step - accuracy: 0.7377 - loss: 6444.3701 - val_accuracy: 0.4169 - val_loss: 305747.0312
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.46319

Epoch 3: val_accuracy did not improve from 0.46319
726/726 - 3s - 4ms/step - accuracy: 0.7389 - loss: 4667.7578 - val_accuracy: 0.4470 - val_loss: 227000.1406
Epoch 4/500

Epoch 4: val_accuracy improved from 0.46319 to 0.46595, saving model to /kaggle/working/checkpoint_model.keras

Epoch 4: val_accuracy improved from 0.46319 to 0.46595, saving model to /kagg

In [42]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_7,  # Features from CICIDS2017
    y_train_resampled_7,  # Labels from CICIDS2017
    validation_data=(X_val_7, y_val_7),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.54783

Epoch 1: val_accuracy did not improve from 0.54783
726/726 - 3s - 5ms/step - accuracy: 0.7657 - loss: 3323.9846 - val_accuracy: 0.3935 - val_loss: 105833.1406
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.54783

Epoch 2: val_accuracy did not improve from 0.54783
726/726 - 3s - 4ms/step - accuracy: 0.7655 - loss: 2694.0835 - val_accuracy: 0.3822 - val_loss: 101285.5078
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.54783

Epoch 3: val_accuracy did not improve from 0.54783
726/726 - 3s - 4ms/step - accuracy: 0.7671 - loss: 2452.0298 - val_accuracy: 0.3940 - val_loss: 94147.3906
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.54783

Epoch 4: val_accuracy did not improve from 0.54783
726/726 - 3s - 4ms/step - accuracy: 0.7693 - loss: 2276.4482 - val_accuracy: 0.4215 - val_loss: 86919.5234
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.54783

Epoch 5: val_accuracy did not improve fro

In [43]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_8,  # Features from CICIDS2017
    y_train_resampled_8,  # Labels from CICIDS2017
    validation_data=(X_val_8, y_val_8),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7635 - loss: 2205.1387 - val_accuracy: 0.4053 - val_loss: 45991.0273
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7628 - loss: 1921.6700 - val_accuracy: 0.4776 - val_loss: 42999.7344
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.57417
726/726 - 3s - 5ms/step - accuracy: 0.7643 - loss: 1841.3839 - val_accuracy: 0.4772 - val_loss: 44651.3555
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7707 - loss: 1721.1205 - val_accuracy: 0.4596 - val_loss: 47325.7305
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7759 - loss: 1728.0625 - val_accuracy: 0.3812 - val_loss: 48638.0391
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7810 - loss: 1647.9293 - val_accuracy: 0.43

In [7]:
# Initial split into training and test sets
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    X, y, test_size=0.3, random_state=44, stratify=y
)

# Further split the training set into training and validation sets
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(
    X_train_2, y_train_2, test_size=0.2, random_state=44, stratify=y_train_2
)

# Check for extremely large values
print("Max value in X_train_2:", np.max(X_train_2))
print("Min value in X_train_2:", np.min(X_train_2))

X_train_2_scaled = scaler.fit_transform(X_train_2)
# Get the original class distribution
class_counts_2 = Counter(y_train_2)
print("Original class distribution:", class_counts_2)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_2 = class_counts_2[min(class_counts_2, key=class_counts_2.get)]
desired_majority_size_2 = minority_class_size_2 * 5

# Create the sampling strategy dictionary
sampling_strategy_2 = {0: desired_majority_size_2, 1: minority_class_size_2}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_2 = RandomUnderSampler(sampling_strategy=sampling_strategy_2, random_state=42)
X_resampled_2, y_resampled_2 = undersampler_2.fit_resample(X_train_2, y_train_2)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_2))

# Apply SMOTE on the smaller subset
X_train_resampled_2, y_train_resampled_2 = smote.fit_resample(X_resampled_2, y_resampled_2)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_2))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_2))

Max value in X_train_2: 2071000000.0
Min value in X_train_2: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [46]:
# Initial split into training and test sets
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    X, y, test_size=0.3, random_state=43, stratify=y
)

# Further split the training set into training and validation sets
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(
    X_train_1, y_train_1, test_size=0.2, random_state=43, stratify=y_train_1
)
# Check for extremely large values
print("Max value in X_train_1:", np.max(X_train_1))
print("Min value in X_train_1:", np.min(X_train_1))

X_train_1_scaled = scaler.fit_transform(X_train_1)

# Get the original class distribution
class_counts_1 = Counter(y_train_1)
print("Original class distribution:", class_counts_1)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_1 = class_counts_1[min(class_counts_1, key=class_counts_1.get)]
desired_majority_size_1 = minority_class_size_1 * 5

# Create the sampling strategy dictionary
sampling_strategy_1 = {0: desired_majority_size_1, 1: minority_class_size_1}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_1 = RandomUnderSampler(sampling_strategy=sampling_strategy_1, random_state=42)
X_resampled_1, y_resampled_1 = undersampler_1.fit_resample(X_train_1, y_train_1)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_1))

# Apply SMOTE on the smaller subset
X_train_resampled_1, y_train_resampled_1 = smote.fit_resample(X_resampled_1, y_resampled_1)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_1))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_1))



Max value in X_train_1: 2071000000.0
Min value in X_train_1: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [47]:
# Initial split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Check for extremely large values
print("Max value in X_train:", np.max(X_train))
print("Min value in X_train:", np.min(X_train))

X_train_scaled = scaler.fit_transform(X_train)

# Get the original class distribution
class_counts = Counter(y_train)
print("Original class distribution:", class_counts)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size = class_counts[min(class_counts, key=class_counts.get)]
desired_majority_size = minority_class_size * 5

# Create the sampling strategy dictionary
sampling_strategy = {0: desired_majority_size, 1: minority_class_size}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled))

X_train_resampled, y_train_resampled = smote.fit_resample(X_resampled, y_resampled)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled))
print("Class Distribution After SMOTE:", Counter(y_train_resampled))

Max value in X_train: 2071000000.0
Min value in X_train: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [48]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled,  # Features from CICIDS2017
    y_train_resampled,  # Labels from CICIDS2017
    validation_data=(X_val, y_val),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)



Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.57417

Epoch 1: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7682 - loss: 2162.2964 - val_accuracy: 0.4420 - val_loss: 64830.5117
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.57417

Epoch 2: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7663 - loss: 1944.3076 - val_accuracy: 0.3951 - val_loss: 64002.0820
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.57417

Epoch 3: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7698 - loss: 1746.6827 - val_accuracy: 0.4060 - val_loss: 73572.6406
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.57417

Epoch 4: val_accuracy did not improve from 0.57417
726/726 - 3s - 4ms/step - accuracy: 0.7702 - loss: 1742.0850 - val_accuracy: 0.3506 - val_loss: 101007.4688
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.57417

Epoch 5: val_accuracy did not improve from

In [49]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_1,  # Features from CICIDS2017
    y_train_resampled_1,  # Labels from CICIDS2017
    validation_data=(X_val_1, y_val_1),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
) 

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.60958

Epoch 1: val_accuracy did not improve from 0.60958
726/726 - 3s - 5ms/step - accuracy: 0.7313 - loss: 4008.8196 - val_accuracy: 0.3091 - val_loss: 750556.5625
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.60958

Epoch 2: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.7763 - loss: 2117.1069 - val_accuracy: 0.4479 - val_loss: 714091.4375
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.60958

Epoch 3: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.7886 - loss: 1779.0482 - val_accuracy: 0.5136 - val_loss: 706495.9375
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.60958

Epoch 4: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.8054 - loss: 1549.9799 - val_accuracy: 0.4938 - val_loss: 710927.5000
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.60958

Epoch 5: val_accuracy did not improve f

In [50]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_2,  # Features from CICIDS2017
    y_train_resampled_2,  # Labels from CICIDS2017
    validation_data=(X_val_2, y_val_2),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.60958

Epoch 1: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.7839 - loss: 56908.4766 - val_accuracy: 0.5737 - val_loss: 2486580.2500
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.60958

Epoch 2: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.7926 - loss: 51747.8906 - val_accuracy: 0.5908 - val_loss: 2330295.0000
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.60958

Epoch 3: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.7908 - loss: 48927.6836 - val_accuracy: 0.5554 - val_loss: 2204088.7500
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.60958

Epoch 4: val_accuracy did not improve from 0.60958
726/726 - 3s - 4ms/step - accuracy: 0.7892 - loss: 46608.1406 - val_accuracy: 0.5309 - val_loss: 2073875.3750
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.60958

Epoch 5: val_accuracy did not i

In [9]:
# Initial split into training and test sets
X_train_9, X_test_9, y_train_9, y_test_9 = train_test_split(
    X, y, test_size=0.3, random_state=51, stratify=y
)

# Further split the training set into training and validation sets
X_train_9, X_val_9, y_train_9, y_val_9 = train_test_split(
    X_train_9, y_train_9, test_size=0.2, random_state=51, stratify=y_train_9
)

# Check for extremely large values
print("Max value in X_train_9:", np.max(X_train_9))
print("Min value in X_train_9:", np.min(X_train_9))

X_train_9_scaled = scaler.fit_transform(X_train_9)

# Get the original class distribution
class_counts_9 = Counter(y_train_9)
print("Original class distribution:", class_counts_9)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_9 = class_counts_9[min(class_counts_9, key=class_counts_9.get)]
desired_majority_size_9 = minority_class_size_9 * 5

# Create the sampling strategy dictionary
sampling_strategy_9 = {0: desired_majority_size_9, 1: minority_class_size_9}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_9 = RandomUnderSampler(sampling_strategy=sampling_strategy_9, random_state=42)
X_resampled_9, y_resampled_9 = undersampler_9.fit_resample(X_train_9, y_train_9)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_9))

# Apply SMOTE on the smaller subset
X_train_resampled_9, y_train_resampled_9 = smote.fit_resample(X_resampled_9, y_resampled_9)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_9))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_9))




Max value in X_train_9: 2071000000.0
Min value in X_train_9: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [10]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_9,  # Features from CICIDS2017
    y_train_resampled_9,  # Labels from CICIDS2017
    validation_data=(X_val_9, y_val_9),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy improved from -inf to 0.70615, saving model to /kaggle/working/checkpoint_model.keras

Epoch 1: val_accuracy improved from -inf to 0.70615, saving model to /kaggle/working/checkpoint_model.weights.h5
726/726 - 3s - 5ms/step - accuracy: 0.7123 - loss: 150820.4219 - val_accuracy: 0.7061 - val_loss: 114762.1641
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.70615

Epoch 2: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7304 - loss: 142205.1562 - val_accuracy: 0.6872 - val_loss: 123395.3750
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.70615

Epoch 3: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7506 - loss: 137414.4062 - val_accuracy: 0.6893 - val_loss: 118842.2891
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.70615

Epoch 4: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7534 - loss: 134142.2969 - val_accuracy: 0.6706 -

In [11]:
# Initial split into training and test sets
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(
    X, y, test_size=0.3, random_state=52, stratify=y
)

# Further split the training set into training and validation sets
X_train_10, X_val_10, y_train_10, y_val_10 = train_test_split(
    X_train_10, y_train_10, test_size=0.2, random_state=52, stratify=y_train_10
)

# Check for extremely large values
print("Max value in X_train_10:", np.max(X_train_10))
print("Min value in X_train_10:", np.min(X_train_10))

X_train_10_scaled = scaler.fit_transform(X_train_10)

# Get the original class distribution
class_counts_10 = Counter(y_train_10)
print("Original class distribution:", class_counts_10)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_10 = class_counts_10[min(class_counts_10, key=class_counts_10.get)]
desired_majority_size_10 = minority_class_size_10 * 5

# Create the sampling strategy dictionary
sampling_strategy_10 = {0: desired_majority_size_10, 1: minority_class_size_10}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_10 = RandomUnderSampler(sampling_strategy=sampling_strategy_10, random_state=42)
X_resampled_10, y_resampled_10 = undersampler_10.fit_resample(X_train_10, y_train_10)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_10))

# Apply SMOTE on the smaller subset
X_train_resampled_10, y_train_resampled_10 = smote.fit_resample(X_resampled_10, y_resampled_10)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_10))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_10))




Max value in X_train_10: 2071000000.0
Min value in X_train_10: -6442447920.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [12]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_10,  # Features from CICIDS2017
    y_train_resampled_10,  # Labels from CICIDS2017
    validation_data=(X_val_10, y_val_10),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.70615

Epoch 1: val_accuracy did not improve from 0.70615
726/726 - 3s - 5ms/step - accuracy: 0.7746 - loss: 1932.5040 - val_accuracy: 0.5506 - val_loss: 48683.1914
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.70615

Epoch 2: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7750 - loss: 1654.7644 - val_accuracy: 0.5242 - val_loss: 47957.6445
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.70615

Epoch 3: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7758 - loss: 1583.7163 - val_accuracy: 0.4550 - val_loss: 51194.9727
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.70615

Epoch 4: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7792 - loss: 1549.1018 - val_accuracy: 0.4667 - val_loss: 51187.2266
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.70615

Epoch 5: val_accuracy did not improve from 

In [13]:
# Initial split into training and test sets
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(
    X, y, test_size=0.3, random_state=53, stratify=y
)

# Further split the training set into training and validation sets
X_train_11, X_val_11, y_train_11, y_val_11 = train_test_split(
    X_train_11, y_train_11, test_size=0.2, random_state=53, stratify=y_train_11
)

# Check for extremely large values
print("Max value in X_train_11:", np.max(X_train_11))
print("Min value in X_train_11:", np.min(X_train_11))

X_train_11_scaled = scaler.fit_transform(X_train_11)

# Get the original class distribution
class_counts_11 = Counter(y_train_11)
print("Original class distribution:", class_counts_11)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_11 = class_counts_11[min(class_counts_11, key=class_counts_11.get)]
desired_majority_size_11 = minority_class_size_11 * 5

# Create the sampling strategy dictionary
sampling_strategy_11 = {0: desired_majority_size_11, 1: minority_class_size_11}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_11 = RandomUnderSampler(sampling_strategy=sampling_strategy_11, random_state=42)
X_resampled_11, y_resampled_11 = undersampler_11.fit_resample(X_train_11, y_train_11)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_11))

# Apply SMOTE on the smaller subset
X_train_resampled_11, y_train_resampled_11 = smote.fit_resample(X_resampled_11, y_resampled_11)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_11))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_11))




Max value in X_train_11: 2071000000.0
Min value in X_train_11: -6442447920.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [14]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_11,  # Features from CICIDS2017
    y_train_resampled_11,  # Labels from CICIDS2017
    validation_data=(X_val_11, y_val_11),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.70615

Epoch 1: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7529 - loss: 3481.9343 - val_accuracy: 0.3779 - val_loss: 48751.2695
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.70615

Epoch 2: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7685 - loss: 2668.8755 - val_accuracy: 0.3193 - val_loss: 52568.8125
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.70615

Epoch 3: val_accuracy did not improve from 0.70615
726/726 - 3s - 5ms/step - accuracy: 0.7699 - loss: 2304.2593 - val_accuracy: 0.3257 - val_loss: 40888.2773
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.70615

Epoch 4: val_accuracy did not improve from 0.70615
726/726 - 3s - 5ms/step - accuracy: 0.7709 - loss: 2120.2703 - val_accuracy: 0.4523 - val_loss: 38645.0664
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.70615

Epoch 5: val_accuracy did not improve from 

In [15]:
# Initial split into training and test sets
X_train_12, X_test_12, y_train_12, y_test_12 = train_test_split(
    X, y, test_size=0.3, random_state=54, stratify=y
)

# Further split the training set into training and validation sets
X_train_12, X_val_12, y_train_12, y_val_12 = train_test_split(
    X_train_12, y_train_12, test_size=0.2, random_state=54, stratify=y_train_12
)

# Check for extremely large values
print("Max value in X_train_12:", np.max(X_train_12))
print("Min value in X_train_12:", np.min(X_train_12))

X_train_12_scaled = scaler.fit_transform(X_train_12)

# Get the original class distribution
class_counts_12 = Counter(y_train_12)
print("Original class distribution:", class_counts_12)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_12 = class_counts_12[min(class_counts_12, key=class_counts_12.get)]
desired_majority_size_12 = minority_class_size_12 * 5

# Create the sampling strategy dictionary
sampling_strategy_12 = {0: desired_majority_size_12, 1: minority_class_size_12}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_12 = RandomUnderSampler(sampling_strategy=sampling_strategy_12, random_state=42)
X_resampled_12, y_resampled_12 = undersampler_12.fit_resample(X_train_12, y_train_12)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_12))

# Apply SMOTE on the smaller subset
X_train_resampled_12, y_train_resampled_12 = smote.fit_resample(X_resampled_12, y_resampled_12)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_12))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_12))




Max value in X_train_12: 2071000000.0
Min value in X_train_12: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [16]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_12,  # Features from CICIDS2017
    y_train_resampled_12,  # Labels from CICIDS2017
    validation_data=(X_val_12, y_val_12),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.70615

Epoch 1: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7270 - loss: 48337.7500 - val_accuracy: 0.3893 - val_loss: 600315.3125
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.70615

Epoch 2: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7454 - loss: 36346.1172 - val_accuracy: 0.3798 - val_loss: 481742.7812
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.70615

Epoch 3: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7442 - loss: 27849.7031 - val_accuracy: 0.3837 - val_loss: 384888.0938
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.70615

Epoch 4: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7487 - loss: 20494.3301 - val_accuracy: 0.3879 - val_loss: 282843.4375
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.70615

Epoch 5: val_accuracy did not impro

In [17]:
# Initial split into training and test sets
X_train_13, X_test_13, y_train_13, y_test_13 = train_test_split(
    X, y, test_size=0.3, random_state=55, stratify=y
)

# Further split the training set into training and validation sets
X_train_13, X_val_13, y_train_13, y_val_13 = train_test_split(
    X_train_13, y_train_13, test_size=0.2, random_state=55, stratify=y_train_13
)

# Check for extremely large values
print("Max value in X_train_13:", np.max(X_train_13))
print("Min value in X_train_13:", np.min(X_train_13))

X_train_13_scaled = scaler.fit_transform(X_train_13)

# Get the original class distribution
class_counts_13 = Counter(y_train_13)
print("Original class distribution:", class_counts_13)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_13 = class_counts_13[min(class_counts_13, key=class_counts_13.get)]
desired_majority_size_13 = minority_class_size_13 * 5

# Create the sampling strategy dictionary
sampling_strategy_13 = {0: desired_majority_size_13, 1: minority_class_size_13}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_13 = RandomUnderSampler(sampling_strategy=sampling_strategy_13, random_state=42)
X_resampled_13, y_resampled_13 = undersampler_13.fit_resample(X_train_13, y_train_13)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_13))

# Apply SMOTE on the smaller subset
X_train_resampled_13, y_train_resampled_13 = smote.fit_resample(X_resampled_13, y_resampled_13)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_13))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_13))




Max value in X_train_13: 2071000000.0
Min value in X_train_13: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [18]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_13,  # Features from CICIDS2017
    y_train_resampled_13,  # Labels from CICIDS2017
    validation_data=(X_val_13, y_val_13),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.70615

Epoch 1: val_accuracy did not improve from 0.70615
726/726 - 3s - 5ms/step - accuracy: 0.7458 - loss: 9535.1904 - val_accuracy: 0.4696 - val_loss: 38463.9023
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.70615

Epoch 2: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7527 - loss: 7136.0254 - val_accuracy: 0.4640 - val_loss: 45853.2812
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.70615

Epoch 3: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7549 - loss: 6504.4727 - val_accuracy: 0.4530 - val_loss: 48122.4297
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.70615

Epoch 4: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7548 - loss: 5918.5532 - val_accuracy: 0.4703 - val_loss: 49135.0430
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.70615

Epoch 5: val_accuracy did not improve from 

In [19]:
# Evaluate the model on the test set
test_loss, test_accuracy = fine_tuned_model.evaluate(X_test_2, y_test_2, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

23633/23633 - 31s - 1ms/step - accuracy: 0.5619 - loss: 43699.6602
Test Loss: 43699.66015625
Test Accuracy: 0.5618956685066223


In [20]:
# Initial split into training and test sets
X_train_14, X_test_14, y_train_14, y_test_14 = train_test_split(
    X, y, test_size=0.3, random_state=56, stratify=y
)

# Further split the training set into training and validation sets
X_train_14, X_val_14, y_train_14, y_val_14 = train_test_split(
    X_train_14, y_train_14, test_size=0.2, random_state=56, stratify=y_train_14
)

# Check for extremely large values
print("Max value in X_train_14:", np.max(X_train_14))
print("Min value in X_train_14:", np.min(X_train_14))

X_train_14_scaled = scaler.fit_transform(X_train_14)

# Get the original class distribution
class_counts_14 = Counter(y_train_14)
print("Original class distribution:", class_counts_14)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_14 = class_counts_14[min(class_counts_14, key=class_counts_14.get)]
desired_majority_size_14 = minority_class_size_14 * 5

# Create the sampling strategy dictionary
sampling_strategy_14 = {0: desired_majority_size_14, 1: minority_class_size_14}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_14 = RandomUnderSampler(sampling_strategy=sampling_strategy_14, random_state=42)
X_resampled_14, y_resampled_14 = undersampler_14.fit_resample(X_train_14, y_train_14)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_14))

# Apply SMOTE on the smaller subset
X_train_resampled_14, y_train_resampled_14 = smote.fit_resample(X_resampled_14, y_resampled_14)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_14))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_14))




Max value in X_train_14: 2071000000.0
Min value in X_train_14: -6442447920.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [21]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_14,  # Features from CICIDS2017
    y_train_resampled_14,  # Labels from CICIDS2017
    validation_data=(X_val_14, y_val_14),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.70615

Epoch 1: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7931 - loss: 1260.8984 - val_accuracy: 0.5678 - val_loss: 50469.1680
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.70615

Epoch 2: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.7972 - loss: 1115.8988 - val_accuracy: 0.5310 - val_loss: 49137.2383
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.70615

Epoch 3: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.8007 - loss: 1102.0013 - val_accuracy: 0.6009 - val_loss: 42674.0938
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.70615

Epoch 4: val_accuracy did not improve from 0.70615
726/726 - 3s - 4ms/step - accuracy: 0.8016 - loss: 1123.8151 - val_accuracy: 0.5746 - val_loss: 44766.8945
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.70615

Epoch 5: val_accuracy did not improve from 

In [7]:
# Initial split into training and test sets
X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split(
    X, y, test_size=0.3, random_state=57, stratify=y
)

# Further split the training set into training and validation sets
X_train_15, X_val_15, y_train_15, y_val_15 = train_test_split(
    X_train_15, y_train_15, test_size=0.2, random_state=57, stratify=y_train_15
)

# Check for extremely large values
print("Max value in X_train_15:", np.max(X_train_15))
print("Min value in X_train_15:", np.min(X_train_15))

X_train_15_scaled = scaler.fit_transform(X_train_15)

# Get the original class distribution
class_counts_15 = Counter(y_train_15)
print("Original class distribution:", class_counts_15)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_15 = class_counts_15[min(class_counts_15, key=class_counts_15.get)]
desired_majority_size_15 = minority_class_size_15 * 5

# Create the sampling strategy dictionary
sampling_strategy_15 = {0: desired_majority_size_15, 1: minority_class_size_15}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_15 = RandomUnderSampler(sampling_strategy=sampling_strategy_15, random_state=42)
X_resampled_15, y_resampled_15 = undersampler_15.fit_resample(X_train_15, y_train_15)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_15))

# Apply SMOTE on the smaller subset
X_train_resampled_15, y_train_resampled_15 = smote.fit_resample(X_resampled_15, y_resampled_15)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_15))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_15))



Max value in X_train_15: 2071000000.0
Min value in X_train_15: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [21]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_15,  # Features from CICIDS2017
    y_train_resampled_15,  # Labels from CICIDS2017
    validation_data=(X_val_15, y_val_15),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)


Epoch 1/500

Epoch 1: val_accuracy improved from -inf to 0.55236, saving model to /kaggle/working/checkpoint_model.keras

Epoch 1: val_accuracy improved from -inf to 0.55236, saving model to /kaggle/working/checkpoint_model.weights.h5
726/726 - 3s - 4ms/step - accuracy: 0.7844 - loss: 2086.7073 - val_accuracy: 0.5524 - val_loss: 83852.6719
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.55236

Epoch 2: val_accuracy did not improve from 0.55236
726/726 - 3s - 4ms/step - accuracy: 0.7951 - loss: 1648.8855 - val_accuracy: 0.5149 - val_loss: 111985.9531
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.55236

Epoch 3: val_accuracy did not improve from 0.55236
726/726 - 3s - 5ms/step - accuracy: 0.7959 - loss: 1530.1821 - val_accuracy: 0.5252 - val_loss: 124110.2812
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.55236

Epoch 4: val_accuracy did not improve from 0.55236
726/726 - 3s - 4ms/step - accuracy: 0.7983 - loss: 1396.7745 - val_accuracy: 0.5071 - val_loss

In [22]:
# Initial split into training and test sets
X_train_16, X_test_16, y_train_16, y_test_16 = train_test_split(
    X, y, test_size=0.3, random_state=58, stratify=y
)

# Further split the training set into training and validation sets
X_train_16, X_val_16, y_train_16, y_val_16 = train_test_split(
    X_train_16, y_train_16, test_size=0.2, random_state=58, stratify=y_train_16
)

# Check for extremely large values
print("Max value in X_train_16:", np.max(X_train_16))
print("Min value in X_train_16:", np.min(X_train_16))

X_train_16_scaled = scaler.fit_transform(X_train_16)

# Get the original class distribution
class_counts_16 = Counter(y_train_16)
print("Original class distribution:", class_counts_16)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_16 = class_counts_16[min(class_counts_16, key=class_counts_16.get)]
desired_majority_size_16 = minority_class_size_16 * 5

# Create the sampling strategy dictionary
sampling_strategy_16 = {0: desired_majority_size_16, 1: minority_class_size_16}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_16 = RandomUnderSampler(sampling_strategy=sampling_strategy_16, random_state=42)
X_resampled_16, y_resampled_16 = undersampler_16.fit_resample(X_train_16, y_train_16)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_16))

# Apply SMOTE on the smaller subset
X_train_resampled_16, y_train_resampled_16 = smote.fit_resample(X_resampled_16, y_resampled_16)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_16))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_16))



Max value in X_train_16: 2071000000.0
Min value in X_train_16: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [23]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_16,  # Features from CICIDS2017
    y_train_resampled_16,  # Labels from CICIDS2017
    validation_data=(X_val_16, y_val_16),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint, model_checkpoint_weight]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.55236

Epoch 1: val_accuracy did not improve from 0.55236
726/726 - 3s - 5ms/step - accuracy: 0.7106 - loss: 4509.8765 - val_accuracy: 0.2337 - val_loss: 108484.7344
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.55236

Epoch 2: val_accuracy did not improve from 0.55236
726/726 - 3s - 5ms/step - accuracy: 0.7256 - loss: 2034.4189 - val_accuracy: 0.2642 - val_loss: 109515.3906
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.55236

Epoch 3: val_accuracy did not improve from 0.55236
726/726 - 3s - 4ms/step - accuracy: 0.7336 - loss: 1634.9445 - val_accuracy: 0.2449 - val_loss: 115297.8047
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.55236

Epoch 4: val_accuracy did not improve from 0.55236
726/726 - 3s - 4ms/step - accuracy: 0.7430 - loss: 1519.4501 - val_accuracy: 0.2894 - val_loss: 107536.5312
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.55236

Epoch 5: val_accuracy did not improve f

In [9]:
# Initial split into training and test sets
X_train_16, X_test_16, y_train_16, y_test_16 = train_test_split(
    X, y, test_size=0.3, random_state=58, stratify=y
)

# Further split the training set into training and validation sets
X_train_16, X_val_16, y_train_16, y_val_16 = train_test_split(
    X_train_16, y_train_16, test_size=0.2, random_state=58, stratify=y_train_16
)

# Check for extremely large values
print("Max value in X_train_16:", np.max(X_train_16))
print("Min value in X_train_16:", np.min(X_train_16))

X_train_16_scaled = scaler.fit_transform(X_train_16)

# Get the original class distribution
class_counts_16 = Counter(y_train_16)
print("Original class distribution:", class_counts_16)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_16 = class_counts_16[min(class_counts_16, key=class_counts_16.get)]
desired_majority_size_16 = minority_class_size_16 * 5

# Create the sampling strategy dictionary
sampling_strategy_16 = {0: desired_majority_size_16, 1: minority_class_size_16}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_16 = RandomUnderSampler(sampling_strategy=sampling_strategy_16, random_state=42)
X_resampled_16, y_resampled_16 = undersampler_16.fit_resample(X_train_16, y_train_16)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_16))

# Apply SMOTE on the smaller subset
X_train_resampled_16, y_train_resampled_16 = smote.fit_resample(X_resampled_16, y_resampled_16)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_16))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_16))


Max value in X_train_16: 2071000000.0
Min value in X_train_16: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [10]:
# ## It Works
# from keras.callbacks import Callback
# from sklearn.metrics import f1_score, roc_auc_score
# from sklearn.preprocessing import label_binarize
# class PerformanceMetrics(Callback):
#     def __init__(self, validation_data, num_classes):
#         super(PerformanceMetrics, self).__init__()
#         self.validation_data = validation_data
#         self.num_classes = num_classes

#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         x_val, y_val = self.validation_data
#         y_pred = self.model.predict(x_val)

#         # Convert predictions and true labels to the correct format
#         y_pred_labels = np.argmax(y_pred, axis=1)  # Convert to labels
#         if len(y_val.shape) == 2:  # Checking if y_val is one-hot encoded
#             y_val = np.argmax(y_val, axis=1)

#         # Calculate F1 Score (weighted can handle class imbalance)
#         f1 = f1_score(y_val, y_pred_labels, average='weighted')
#         logs['val_f1_score'] = f1

#         # Prepare ROC-AUC for multiclass
#         if self.num_classes > 2:
#             y_val_bin = label_binarize(y_val, classes=np.arange(self.num_classes))
#             y_pred_bin = label_binarize(y_pred_labels, classes=np.arange(self.num_classes))
#             roc_auc = roc_auc_score(y_val_bin, y_pred, average='macro', multi_class='ovr')
#         else:
#             roc_auc = roc_auc_score(y_val, y_pred[:, 1])

#         logs['val_roc_auc'] = roc_auc
#         print(f"Epoch {epoch+1}: val_f1_score = {f1:.4f}, val_roc_auc = {roc_auc:.4f}")


# model_checkpoint_f1 = ModelCheckpoint(
#     filepath='/kaggle/working/checkpoint_model_f1_gru.keras',
#     save_weights_only=False,
#     monitor='val_f1_score',
#     mode='max',
#     save_best_only=True,
#     verbose=1
# )

# model_checkpoint_roc_auc = ModelCheckpoint(
#     filepath='/kaggle/working/checkpoint_model_roc_auc_gru.keras',
#     save_weights_only=False,
#     monitor='val_roc_auc',
#     mode='max',
#     save_best_only=True,
#     verbose=1
# )




In [11]:
## It works
# # Initialize the performance metrics callback
# performance_metrics = PerformanceMetrics(validation_data=(X_val_16, y_val_16), num_classes = 15)

# # Fit the model
# history = fine_tuned_model.fit(
#     X_train_resampled_16,
#     y_train_resampled_16,
#     validation_data=(X_val_16, y_val_16),
#     epochs=500,
#     batch_size=2000,
#     verbose=2,
#     callbacks=[model_checkpoint, performance_metrics, model_checkpoint_f1, model_checkpoint_roc_auc]
# )



# # Train the fine-tuned model
# history = fine_tuned_model.fit(
#     X_train_resampled_16,  # Features from CICIDS2017
#     y_train_resampled_16,  # Labels from CICIDS2017
#     validation_data=(X_val_16, y_val_16),  # Validation set
#     epochs=500,  # Adjust based on the dataset size
#     batch_size=2000,  # Adjust batch size as needed
#     verbose=2,
#     callbacks=[model_checkpoint,  model_checkpoint_f1, model_checkpoint_roc_auc]
# )


In [12]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_16,  # Features from CICIDS2017
    y_train_resampled_16,  # Labels from CICIDS2017
    validation_data=(X_val_16, y_val_16),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy improved from -inf to 0.33562, saving model to /kaggle/working/checkpoint_model.keras
726/726 - 4s - 5ms/step - accuracy: 0.7638 - loss: 3742.0969 - val_accuracy: 0.3356 - val_loss: 167009.4062
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.33562
726/726 - 3s - 4ms/step - accuracy: 0.7548 - loss: 3064.7405 - val_accuracy: 0.3316 - val_loss: 191368.8906
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.33562
726/726 - 3s - 4ms/step - accuracy: 0.7497 - loss: 2844.6311 - val_accuracy: 0.3320 - val_loss: 215046.8750
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.33562
726/726 - 3s - 4ms/step - accuracy: 0.7492 - loss: 2641.4243 - val_accuracy: 0.3197 - val_loss: 225183.0469
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.33562
726/726 - 3s - 4ms/step - accuracy: 0.7474 - loss: 2563.9451 - val_accuracy: 0.2989 - val_loss: 248038.8438
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.33562
726/726 - 3s - 4ms/

In [13]:
# Initial split into training and test sets
X_train_17, X_test_17, y_train_17, y_test_17 = train_test_split(
    X, y, test_size=0.3, random_state=59, stratify=y
)

# Further split the training set into training and validation sets
X_train_17, X_val_17, y_train_17, y_val_17 = train_test_split(
    X_train_17, y_train_17, test_size=0.2, random_state=59, stratify=y_train_17
)

# Check for extremely large values
print("Max value in X_train_17:", np.max(X_train_17))
print("Min value in X_train_17:", np.min(X_train_17))

X_train_17_scaled = scaler.fit_transform(X_train_17)

# Get the original class distribution
class_counts_17 = Counter(y_train_17)
print("Original class distribution:", class_counts_17)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_17 = class_counts_17[min(class_counts_17, key=class_counts_17.get)]
desired_majority_size_17 = minority_class_size_17 * 5

# Create the sampling strategy dictionary
sampling_strategy_17 = {0: desired_majority_size_17, 1: minority_class_size_17}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_17 = RandomUnderSampler(sampling_strategy=sampling_strategy_17, random_state=42)
X_resampled_17, y_resampled_17 = undersampler_17.fit_resample(X_train_17, y_train_17)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_17))

# Apply SMOTE on the smaller subset
X_train_resampled_17, y_train_resampled_17 = smote.fit_resample(X_resampled_17, y_resampled_17)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_17))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_17))



Max value in X_train_17: 2071000000.0
Min value in X_train_17: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [14]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_17,  # Features from CICIDS2017
    y_train_resampled_17,  # Labels from CICIDS2017
    validation_data=(X_val_17, y_val_17),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7713 - loss: 2223.1611 - val_accuracy: 0.5567 - val_loss: 88122.5078
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7930 - loss: 1463.0874 - val_accuracy: 0.5584 - val_loss: 85861.8359
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7965 - loss: 1424.8180 - val_accuracy: 0.5380 - val_loss: 86734.3750
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.8035 - loss: 1289.0625 - val_accuracy: 0.5730 - val_loss: 85942.2969
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.8031 - loss: 1366.3988 - val_accuracy: 0.5597 - val_loss: 91431.8828
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.8034 - loss: 1359.2889 - val_accuracy: 0.53

In [15]:
# Initial split into training and test sets
X_train_18, X_test_18, y_train_18, y_test_18 = train_test_split(
    X, y, test_size=0.3, random_state=60, stratify=y
)

# Further split the training set into training and validation sets
X_train_18, X_val_18, y_train_18, y_val_18 = train_test_split(
    X_train_18, y_train_18, test_size=0.2, random_state=60, stratify=y_train_18
)

# Check for extremely large values
print("Max value in X_train_18:", np.max(X_train_18))
print("Min value in X_train_18:", np.min(X_train_18))

X_train_18_scaled = scaler.fit_transform(X_train_18)

# Get the original class distribution
class_counts_18 = Counter(y_train_18)
print("Original class distribution:", class_counts_18)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_18 = class_counts_18[min(class_counts_18, key=class_counts_18.get)]
desired_majority_size_18 = minority_class_size_18 * 5

# Create the sampling strategy dictionary
sampling_strategy_18 = {0: desired_majority_size_18, 1: minority_class_size_18}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_18 = RandomUnderSampler(sampling_strategy=sampling_strategy_18, random_state=42)
X_resampled_18, y_resampled_18 = undersampler_18.fit_resample(X_train_18, y_train_18)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_18))

# Apply SMOTE on the smaller subset
X_train_resampled_18, y_train_resampled_18 = smote.fit_resample(X_resampled_18, y_resampled_18)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_18))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_18))



Max value in X_train_18: 2071000000.0
Min value in X_train_18: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [16]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_18,  # Features from CICIDS2017
    y_train_resampled_18,  # Labels from CICIDS2017
    validation_data=(X_val_18, y_val_18),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.60639
726/726 - 4s - 5ms/step - accuracy: 0.7760 - loss: 2137.9412 - val_accuracy: 0.5473 - val_loss: 91661.1172
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7942 - loss: 1555.3724 - val_accuracy: 0.5550 - val_loss: 92621.2188
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.60639
726/726 - 3s - 5ms/step - accuracy: 0.7950 - loss: 1557.3551 - val_accuracy: 0.5844 - val_loss: 87352.7969
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.60639
726/726 - 3s - 5ms/step - accuracy: 0.8006 - loss: 1371.1947 - val_accuracy: 0.5839 - val_loss: 84236.7109
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7965 - loss: 1458.5094 - val_accuracy: 0.5602 - val_loss: 83695.5703
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.8002 - loss: 1312.6398 - val_accuracy: 0.55

In [17]:
# Initial split into training and test sets
X_train_19, X_test_19, y_train_19, y_test_19 = train_test_split(
    X, y, test_size=0.3, random_state=61, stratify=y
)

# Further split the training set into training and validation sets
X_train_19, X_val_19, y_train_19, y_val_19 = train_test_split(
    X_train_19, y_train_19, test_size=0.2, random_state=61, stratify=y_train_19
)

# Check for extremely large values
print("Max value in X_train_19:", np.max(X_train_19))
print("Min value in X_train_19:", np.min(X_train_19))

X_train_19_scaled = scaler.fit_transform(X_train_19)

# Get the original class distribution
class_counts_19 = Counter(y_train_19)
print("Original class distribution:", class_counts_19)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_19 = class_counts_19[min(class_counts_19, key=class_counts_19.get)]
desired_majority_size_19 = minority_class_size_19 * 5

# Create the sampling strategy dictionary
sampling_strategy_19 = {0: desired_majority_size_19, 1: minority_class_size_19}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_19 = RandomUnderSampler(sampling_strategy=sampling_strategy_19, random_state=42)
X_resampled_19, y_resampled_19 = undersampler_19.fit_resample(X_train_19, y_train_19)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_19))

# Apply SMOTE on the smaller subset
X_train_resampled_19, y_train_resampled_19 = smote.fit_resample(X_resampled_19, y_resampled_19)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_19))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_19))




Max value in X_train_19: 2071000000.0
Min value in X_train_19: -6442447920.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [18]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_19,  # Features from CICIDS2017
    y_train_resampled_19,  # Labels from CICIDS2017
    validation_data=(X_val_19, y_val_19),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.60639
726/726 - 3s - 5ms/step - accuracy: 0.7853 - loss: 2846.4771 - val_accuracy: 0.4331 - val_loss: 52017.9062
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7966 - loss: 2415.5850 - val_accuracy: 0.4477 - val_loss: 46865.5938
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7978 - loss: 2246.9011 - val_accuracy: 0.4952 - val_loss: 42723.6172
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7970 - loss: 2182.9250 - val_accuracy: 0.4483 - val_loss: 39247.8008
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7972 - loss: 2038.3986 - val_accuracy: 0.3836 - val_loss: 36847.5703
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.60639
726/726 - 3s - 5ms/step - accuracy: 0.7981 - loss: 1852.0981 - val_accuracy: 0.36

In [19]:
# Initial split into training and test sets
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(
    X, y, test_size=0.3, random_state=62, stratify=y
)

# Further split the training set into training and validation sets
X_train_20, X_val_20, y_train_20, y_val_20 = train_test_split(
    X_train_20, y_train_20, test_size=0.2, random_state=62, stratify=y_train_20
)

# Check for extremely large values
print("Max value in X_train_20:", np.max(X_train_20))
print("Min value in X_train_20:", np.min(X_train_20))

X_train_20_scaled = scaler.fit_transform(X_train_20)

# Get the original class distribution
class_counts_20 = Counter(y_train_20)
print("Original class distribution:", class_counts_20)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_20 = class_counts_20[min(class_counts_20, key=class_counts_20.get)]
desired_majority_size_20 = minority_class_size_20 * 5

# Create the sampling strategy dictionary
sampling_strategy_20 = {0: desired_majority_size_20, 1: minority_class_size_20}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_20 = RandomUnderSampler(sampling_strategy=sampling_strategy_20, random_state=42)
X_resampled_20, y_resampled_20 = undersampler_20.fit_resample(X_train_20, y_train_20)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_20))

# Apply SMOTE on the smaller subset
X_train_resampled_20, y_train_resampled_20 = smote.fit_resample(X_resampled_20, y_resampled_20)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_20))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_20))




Max value in X_train_20: 2071000000.0
Min value in X_train_20: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [20]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_20,  # Features from CICIDS2017
    y_train_resampled_20,  # Labels from CICIDS2017
    validation_data=(X_val_20, y_val_20),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7551 - loss: 72116.0234 - val_accuracy: 0.3882 - val_loss: 22070.2266
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7541 - loss: 70381.9766 - val_accuracy: 0.3645 - val_loss: 28529.8379
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7525 - loss: 69245.3672 - val_accuracy: 0.3821 - val_loss: 29089.4590
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7528 - loss: 68001.3672 - val_accuracy: 0.3835 - val_loss: 32249.7246
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.60639
726/726 - 3s - 5ms/step - accuracy: 0.7532 - loss: 66851.7266 - val_accuracy: 0.2353 - val_loss: 44433.5234
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.60639
726/726 - 3s - 4ms/step - accuracy: 0.7490 - loss: 65817.6016 - val_accuracy

In [8]:
# Initial split into training and test sets
X_train_21, X_test_21, y_train_21, y_test_21 = train_test_split(
    X, y, test_size=0.3, random_state=63, stratify=y
)

# Further split the training set into training and validation sets
X_train_21, X_val_21, y_train_21, y_val_21 = train_test_split(
    X_train_21, y_train_21, test_size=0.2, random_state=63, stratify=y_train_21
)

# Check for extremely large values
print("Max value in X_train_21:", np.max(X_train_21))
print("Min value in X_train_21:", np.min(X_train_21))

X_train_21_scaled = scaler.fit_transform(X_train_21)

# Get the original class distribution
class_counts_21 = Counter(y_train_21)
print("Original class distribution:", class_counts_21)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_21 = class_counts_21[min(class_counts_21, key=class_counts_21.get)]
desired_majority_size_21 = minority_class_size_21 * 5

# Create the sampling strategy dictionary
sampling_strategy_21 = {0: desired_majority_size_21, 1: minority_class_size_21}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_21 = RandomUnderSampler(sampling_strategy=sampling_strategy_21, random_state=42)
X_resampled_21, y_resampled_21 = undersampler_21.fit_resample(X_train_21, y_train_21)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_21))

# Apply SMOTE on the smaller subset
X_train_resampled_21, y_train_resampled_21 = smote.fit_resample(X_resampled_21, y_resampled_21)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_21))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_21))




Max value in X_train_21: 2071000000.0
Min value in X_train_21: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [9]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_21,  # Features from CICIDS2017
    y_train_resampled_21,  # Labels from CICIDS2017
    validation_data=(X_val_21, y_val_21),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy improved from -inf to 0.62652, saving model to /kaggle/working/checkpoint_model.keras
726/726 - 4s - 6ms/step - accuracy: 0.7819 - loss: 8472.4424 - val_accuracy: 0.6265 - val_loss: 67132.1875
Epoch 2/500

Epoch 2: val_accuracy improved from 0.62652 to 0.68071, saving model to /kaggle/working/checkpoint_model.keras
726/726 - 3s - 5ms/step - accuracy: 0.7942 - loss: 7068.4238 - val_accuracy: 0.6807 - val_loss: 50704.6133
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.68071
726/726 - 3s - 4ms/step - accuracy: 0.7948 - loss: 6232.9575 - val_accuracy: 0.5757 - val_loss: 50767.5273
Epoch 4/500

Epoch 4: val_accuracy improved from 0.68071 to 0.69325, saving model to /kaggle/working/checkpoint_model.keras
726/726 - 3s - 4ms/step - accuracy: 0.7981 - loss: 5617.0654 - val_accuracy: 0.6932 - val_loss: 41310.5469
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.8036 - loss: 5127.6011 - val_accuracy

In [10]:
# Initial split into training and test sets
X_train_22, X_test_22, y_train_22, y_test_22 = train_test_split(
    X, y, test_size=0.3, random_state=64, stratify=y
)

# Further split the training set into training and validation sets
X_train_22, X_val_22, y_train_22, y_val_22 = train_test_split(
    X_train_22, y_train_22, test_size=0.2, random_state=64, stratify=y_train_22
)

# Check for extremely large values
print("Max value in X_train_22:", np.max(X_train_22))
print("Min value in X_train_22:", np.min(X_train_22))

X_train_22_scaled = scaler.fit_transform(X_train_22)

# Get the original class distribution
class_counts_22 = Counter(y_train_22)
print("Original class distribution:", class_counts_22)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_22 = class_counts_22[min(class_counts_22, key=class_counts_22.get)]
desired_majority_size_22 = minority_class_size_22 * 5

# Create the sampling strategy dictionary
sampling_strategy_22 = {0: desired_majority_size_22, 1: minority_class_size_22}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_22 = RandomUnderSampler(sampling_strategy=sampling_strategy_22, random_state=42)
X_resampled_22, y_resampled_22 = undersampler_22.fit_resample(X_train_22, y_train_22)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_22))

# Apply SMOTE on the smaller subset
X_train_resampled_22, y_train_resampled_22 = smote.fit_resample(X_resampled_22, y_resampled_22)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_22))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_22))




Max value in X_train_22: 2071000000.0
Min value in X_train_22: -6442447920.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [11]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_22,  # Features from CICIDS2017
    y_train_resampled_22,  # Labels from CICIDS2017
    validation_data=(X_val_22, y_val_22),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7659 - loss: 1318.6726 - val_accuracy: 0.3859 - val_loss: 21495.6094
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7758 - loss: 1114.3823 - val_accuracy: 0.3689 - val_loss: 24191.2012
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7807 - loss: 1148.6219 - val_accuracy: 0.4211 - val_loss: 19656.4570
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7841 - loss: 1106.9856 - val_accuracy: 0.3851 - val_loss: 21865.1602
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7870 - loss: 1067.5795 - val_accuracy: 0.2958 - val_loss: 141696.9219
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7880 - loss: 1210.2329 - val_accuracy: 0.3

In [12]:
# Initial split into training and test sets
X_train_23, X_test_23, y_train_23, y_test_23 = train_test_split(
    X, y, test_size=0.3, random_state=65, stratify=y
)

# Further split the training set into training and validation sets
X_train_23, X_val_23, y_train_23, y_val_23 = train_test_split(
    X_train_23, y_train_23, test_size=0.2, random_state=65, stratify=y_train_23
)

# Check for extremely large values
print("Max value in X_train_23:", np.max(X_train_23))
print("Min value in X_train_23:", np.min(X_train_23))

X_train_23_scaled = scaler.fit_transform(X_train_23)

# Get the original class distribution
class_counts_23 = Counter(y_train_23)
print("Original class distribution:", class_counts_23)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_23 = class_counts_23[min(class_counts_23, key=class_counts_23.get)]
desired_majority_size_23 = minority_class_size_23 * 5

# Create the sampling strategy dictionary
sampling_strategy_23 = {0: desired_majority_size_23, 1: minority_class_size_23}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_23 = RandomUnderSampler(sampling_strategy=sampling_strategy_23, random_state=42)
X_resampled_23, y_resampled_23 = undersampler_23.fit_resample(X_train_23, y_train_23)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_23))

# Apply SMOTE on the smaller subset
X_train_resampled_23, y_train_resampled_23 = smote.fit_resample(X_resampled_23, y_resampled_23)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_23))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_23))



Max value in X_train_23: 2071000000.0
Min value in X_train_23: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [13]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_23,  # Features from CICIDS2017
    y_train_resampled_23,  # Labels from CICIDS2017
    validation_data=(X_val_23, y_val_23),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7900 - loss: 3063.0352 - val_accuracy: 0.4603 - val_loss: 33144.4258
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7906 - loss: 2319.8157 - val_accuracy: 0.4068 - val_loss: 28792.6855
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7907 - loss: 2173.1799 - val_accuracy: 0.4347 - val_loss: 26224.2910
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7896 - loss: 2119.3589 - val_accuracy: 0.4796 - val_loss: 22409.3281
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7913 - loss: 2030.0692 - val_accuracy: 0.4095 - val_loss: 22890.5742
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7909 - loss: 2041.8951 - val_accuracy: 0.40

In [14]:
# Initial split into training and test sets
X_train_24, X_test_24, y_train_24, y_test_24 = train_test_split(
    X, y, test_size=0.3, random_state=66, stratify=y
)

# Further split the training set into training and validation sets
X_train_24, X_val_24, y_train_24, y_val_24 = train_test_split(
    X_train_24, y_train_24, test_size=0.2, random_state=66, stratify=y_train_24
)

# Check for extremely large values
print("Max value in X_train_24:", np.max(X_train_24))
print("Min value in X_train_24:", np.min(X_train_24))

X_train_24_scaled = scaler.fit_transform(X_train_24)

# Get the original class distribution
class_counts_24 = Counter(y_train_24)
print("Original class distribution:", class_counts_24)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_24 = class_counts_24[min(class_counts_24, key=class_counts_24.get)]
desired_majority_size_24 = minority_class_size_24 * 5

# Create the sampling strategy dictionary
sampling_strategy_24 = {0: desired_majority_size_24, 1: minority_class_size_24}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_24 = RandomUnderSampler(sampling_strategy=sampling_strategy_24, random_state=42)
X_resampled_24, y_resampled_24 = undersampler_24.fit_resample(X_train_24, y_train_24)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_24))

# Apply SMOTE on the smaller subset
X_train_resampled_24, y_train_resampled_24 = smote.fit_resample(X_resampled_24, y_resampled_24)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_24))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_24))



Max value in X_train_24: 2071000000.0
Min value in X_train_24: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [15]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_24,  # Features from CICIDS2017
    y_train_resampled_24,  # Labels from CICIDS2017
    validation_data=(X_val_24, y_val_24),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7776 - loss: 2126.4905 - val_accuracy: 0.4517 - val_loss: 26009.2793
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7802 - loss: 1547.9546 - val_accuracy: 0.4547 - val_loss: 28453.1699
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7775 - loss: 1369.8917 - val_accuracy: 0.4647 - val_loss: 31712.9219
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7732 - loss: 1351.0094 - val_accuracy: 0.4316 - val_loss: 31642.8047
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7709 - loss: 1359.4233 - val_accuracy: 0.4543 - val_loss: 35057.9141
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7755 - loss: 1264.1509 - val_accuracy: 0.43

In [16]:
# Initial split into training and test sets
X_train_25, X_test_25, y_train_25, y_test_25 = train_test_split(
    X, y, test_size=0.3, random_state=67, stratify=y
)

# Further split the training set into training and validation sets
X_train_25, X_val_25, y_train_25, y_val_25 = train_test_split(
    X_train_25, y_train_25, test_size=0.2, random_state=67, stratify=y_train_25
)

# Check for extremely large values
print("Max value in X_train_25:", np.max(X_train_25))
print("Min value in X_train_25:", np.min(X_train_25))

X_train_25_scaled = scaler.fit_transform(X_train_25)

# Get the original class distribution
class_counts_25 = Counter(y_train_25)
print("Original class distribution:", class_counts_25)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_25 = class_counts_25[min(class_counts_25, key=class_counts_25.get)]
desired_majority_size_25 = minority_class_size_25 * 5

# Create the sampling strategy dictionary
sampling_strategy_25 = {0: desired_majority_size_25, 1: minority_class_size_25}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_25 = RandomUnderSampler(sampling_strategy=sampling_strategy_25, random_state=42)
X_resampled_25, y_resampled_25 = undersampler_25.fit_resample(X_train_25, y_train_25)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_25))

# Apply SMOTE on the smaller subset
X_train_resampled_25, y_train_resampled_25 = smote.fit_resample(X_resampled_25, y_resampled_25)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_25))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_25))




Max value in X_train_25: 2071000000.0
Min value in X_train_25: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [17]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_25,  # Features from CICIDS2017
    y_train_resampled_25,  # Labels from CICIDS2017
    validation_data=(X_val_25, y_val_25),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7743 - loss: 8108.9893 - val_accuracy: 0.4671 - val_loss: 30868.4961
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7973 - loss: 2278.0972 - val_accuracy: 0.5419 - val_loss: 22021.1758
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.8043 - loss: 1841.8060 - val_accuracy: 0.4646 - val_loss: 20722.1367
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.8075 - loss: 1634.4938 - val_accuracy: 0.4501 - val_loss: 16815.8594
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.8052 - loss: 1647.8906 - val_accuracy: 0.4525 - val_loss: 14884.1260
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.8081 - loss: 1483.7769 - val_accuracy: 0.52

In [18]:
# Initial split into training and test sets
X_train_26, X_test_26, y_train_26, y_test_26 = train_test_split(
    X, y, test_size=0.3, random_state=68, stratify=y
)

# Further split the training set into training and validation sets
X_train_26, X_val_26, y_train_26, y_val_26 = train_test_split(
    X_train_26, y_train_26, test_size=0.2, random_state=68, stratify=y_train_26
)

# Check for extremely large values
print("Max value in X_train_26:", np.max(X_train_26))
print("Min value in X_train_26:", np.min(X_train_26))

X_train_26_scaled = scaler.fit_transform(X_train_26)

# Get the original class distribution
class_counts_26 = Counter(y_train_26)
print("Original class distribution:", class_counts_26)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_26 = class_counts_26[min(class_counts_26, key=class_counts_26.get)]
desired_majority_size_26 = minority_class_size_26 * 5

# Create the sampling strategy dictionary
sampling_strategy_26 = {0: desired_majority_size_26, 1: minority_class_size_26}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_26 = RandomUnderSampler(sampling_strategy=sampling_strategy_26, random_state=42)
X_resampled_26, y_resampled_26 = undersampler_26.fit_resample(X_train_26, y_train_26)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_26))

# Apply SMOTE on the smaller subset
X_train_resampled_26, y_train_resampled_26 = smote.fit_resample(X_resampled_26, y_resampled_26)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_26))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_26))




Max value in X_train_26: 2071000000.0
Min value in X_train_26: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [19]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_26,  # Features from CICIDS2017
    y_train_resampled_26,  # Labels from CICIDS2017
    validation_data=(X_val_26, y_val_26),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7451 - loss: 2741.5549 - val_accuracy: 0.4370 - val_loss: 16187.5547
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7663 - loss: 1767.2750 - val_accuracy: 0.4047 - val_loss: 19379.2715
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7700 - loss: 1720.3386 - val_accuracy: 0.4345 - val_loss: 21142.9668
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7757 - loss: 1638.7904 - val_accuracy: 0.3350 - val_loss: 22356.9160
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7748 - loss: 1579.5775 - val_accuracy: 0.3265 - val_loss: 25878.6953
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7761 - loss: 1591.4938 - val_accuracy: 0.39

In [20]:
# Initial split into training and test sets
X_train_27, X_test_27, y_train_27, y_test_27 = train_test_split(
    X, y, test_size=0.3, random_state=69, stratify=y
)

# Further split the training set into training and validation sets
X_train_27, X_val_27, y_train_27, y_val_27 = train_test_split(
    X_train_27, y_train_27, test_size=0.2, random_state=69, stratify=y_train_27
)

# Check for extremely large values
print("Max value in X_train_27:", np.max(X_train_27))
print("Min value in X_train_27:", np.min(X_train_27))

X_train_27_scaled = scaler.fit_transform(X_train_27)

# Get the original class distribution
class_counts_27 = Counter(y_train_27)
print("Original class distribution:", class_counts_27)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_27 = class_counts_27[min(class_counts_27, key=class_counts_27.get)]
desired_majority_size_27 = minority_class_size_27 * 5

# Create the sampling strategy dictionary
sampling_strategy_27 = {0: desired_majority_size_27, 1: minority_class_size_27}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_27 = RandomUnderSampler(sampling_strategy=sampling_strategy_27, random_state=42)
X_resampled_27, y_resampled_27 = undersampler_27.fit_resample(X_train_27, y_train_27)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_27))

# Apply SMOTE on the smaller subset
X_train_resampled_27, y_train_resampled_27 = smote.fit_resample(X_resampled_27, y_resampled_27)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_27))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_27))



Max value in X_train_27: 2071000000.0
Min value in X_train_27: -32212234632.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [21]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_27,  # Features from CICIDS2017
    y_train_resampled_27,  # Labels from CICIDS2017
    validation_data=(X_val_27, y_val_27),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7140 - loss: 138456.1875 - val_accuracy: 0.2855 - val_loss: 3374688.0000
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7235 - loss: 135505.2031 - val_accuracy: 0.2739 - val_loss: 3394514.5000
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7254 - loss: 133432.6719 - val_accuracy: 0.2510 - val_loss: 3411175.5000
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7210 - loss: 131617.2812 - val_accuracy: 0.2226 - val_loss: 3426705.5000
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7201 - loss: 129830.0938 - val_accuracy: 0.2532 - val_loss: 3428916.0000
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7211 - loss: 127979.789

In [22]:
# Initial split into training and test sets
X_train_28, X_test_28, y_train_28, y_test_28 = train_test_split(
    X, y, test_size=0.3, random_state=70, stratify=y
)

# Further split the training set into training and validation sets
X_train_28, X_val_28, y_train_28, y_val_28 = train_test_split(
    X_train_28, y_train_28, test_size=0.2, random_state=70, stratify=y_train_28
)

# Check for extremely large values
print("Max value in X_train_28:", np.max(X_train_28))
print("Min value in X_train_28:", np.min(X_train_28))

X_train_28_scaled = scaler.fit_transform(X_train_28)

# Get the original class distribution
class_counts_28 = Counter(y_train_28)
print("Original class distribution:", class_counts_28)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_28 = class_counts_28[min(class_counts_28, key=class_counts_28.get)]
desired_majority_size_28 = minority_class_size_28 * 5

# Create the sampling strategy dictionary
sampling_strategy_28 = {0: desired_majority_size_28, 1: minority_class_size_28}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_28 = RandomUnderSampler(sampling_strategy=sampling_strategy_28, random_state=42)
X_resampled_28, y_resampled_28 = undersampler_28.fit_resample(X_train_28, y_train_28)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_28))

# Apply SMOTE on the smaller subset
X_train_resampled_28, y_train_resampled_28 = smote.fit_resample(X_resampled_28, y_resampled_28)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_28))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_28))




Max value in X_train_28: 2071000000.0
Min value in X_train_28: -6442447920.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [23]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_28,  # Features from CICIDS2017
    y_train_resampled_28,  # Labels from CICIDS2017
    validation_data=(X_val_28, y_val_28),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.69325
726/726 - 4s - 5ms/step - accuracy: 0.7807 - loss: 163479.7500 - val_accuracy: 0.4273 - val_loss: 4778644.5000
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7899 - loss: 155536.9219 - val_accuracy: 0.4219 - val_loss: 4593787.0000
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.69325
726/726 - 3s - 5ms/step - accuracy: 0.7855 - loss: 150137.5625 - val_accuracy: 0.4442 - val_loss: 4448046.0000
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7826 - loss: 145529.9688 - val_accuracy: 0.4682 - val_loss: 4308395.0000
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7786 - loss: 141512.9844 - val_accuracy: 0.3387 - val_loss: 4192736.2500
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.69325
726/726 - 3s - 4ms/step - accuracy: 0.7739 - loss: 137672.875

In [15]:
# Initial split into training and test sets
X_train_29, X_test_29, y_train_29, y_test_29 = train_test_split(
    X, y, test_size=0.3, random_state=71, stratify=y
)

# Further split the training set into training and validation sets
X_train_29, X_val_29, y_train_29, y_val_29 = train_test_split(
    X_train_29, y_train_29, test_size=0.2, random_state=71, stratify=y_train_29
)

# Check for extremely large values
print("Max value in X_train_29:", np.max(X_train_29))
print("Min value in X_train_29:", np.min(X_train_29))

X_train_29_scaled = scaler.fit_transform(X_train_29)

# Get the original class distribution
class_counts_29 = Counter(y_train_29)
print("Original class distribution:", class_counts_29)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_29 = class_counts_29[min(class_counts_29, key=class_counts_29.get)]
desired_majority_size_29 = minority_class_size_29 * 5

# Create the sampling strategy dictionary
sampling_strategy_29 = {0: desired_majority_size_29, 1: minority_class_size_29}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_29 = RandomUnderSampler(sampling_strategy=sampling_strategy_29, random_state=42)
X_resampled_29, y_resampled_29 = undersampler_29.fit_resample(X_train_29, y_train_29)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_29))

# Apply SMOTE on the smaller subset
X_train_resampled_29, y_train_resampled_29 = smote.fit_resample(X_resampled_29, y_resampled_29)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_29))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_29))



Max value in X_train_29: 2071000000.0
Min value in X_train_29: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [16]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_29,  # Features from CICIDS2017
    y_train_resampled_29,  # Labels from CICIDS2017
    validation_data=(X_val_29, y_val_29),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)


Epoch 1/500

Epoch 1: val_accuracy improved from -inf to 0.64073, saving model to /kaggle/working/checkpoint_model.keras
726/726 - 2s - 3ms/step - accuracy: 0.7780 - loss: 4147.7598 - val_accuracy: 0.6407 - val_loss: 107972.9609
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.7902 - loss: 2986.9075 - val_accuracy: 0.6168 - val_loss: 113434.6250
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.7867 - loss: 2847.3005 - val_accuracy: 0.5858 - val_loss: 127718.7344
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.7817 - loss: 2663.1790 - val_accuracy: 0.5980 - val_loss: 140126.3438
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.7743 - loss: 2579.2810 - val_accuracy: 0.5902 - val_loss: 148720.4688
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/

In [17]:
# Initial split into training and test sets
X_train_30, X_test_30, y_train_30, y_test_30 = train_test_split(
    X, y, test_size=0.3, random_state=72, stratify=y
)

# Further split the training set into training and validation sets
X_train_30, X_val_30, y_train_30, y_val_30 = train_test_split(
    X_train_30, y_train_30, test_size=0.2, random_state=72, stratify=y_train_30
)

# Check for extremely large values
print("Max value in X_train_30:", np.max(X_train_30))
print("Min value in X_train_30:", np.min(X_train_30))

X_train_30_scaled = scaler.fit_transform(X_train_30)

# Get the original class distribution
class_counts_30 = Counter(y_train_30)
print("Original class distribution:", class_counts_30)

# Set the desired ratio: majority class = 5 times the minority class
minority_class_size_30 = class_counts_30[min(class_counts_30, key=class_counts_30.get)]
desired_majority_size_30 = minority_class_size_30 * 5

# Create the sampling strategy dictionary
sampling_strategy_30 = {0: desired_majority_size_30, 1: minority_class_size_30}  # Adjust class labels accordingly

# Apply RandomUnderSampler with the dictionary
undersampler_30 = RandomUnderSampler(sampling_strategy=sampling_strategy_30, random_state=42)
X_resampled_30, y_resampled_30 = undersampler_30.fit_resample(X_train_30, y_train_30)

# Check the new class distribution
print("Class distribution after undersampling:", Counter(y_resampled_30))

# Apply SMOTE on the smaller subset
X_train_resampled_30, y_train_resampled_30 = smote.fit_resample(X_resampled_30, y_resampled_30)


#Verify the class distribution after SMOTE
from collections import Counter
print("Class Distribution Before SMOTE:", Counter(y_resampled_30))
print("Class Distribution After SMOTE:", Counter(y_train_resampled_30))




Max value in X_train_30: 2071000000.0
Min value in X_train_30: -9663668122.0
Original class distribution: Counter({0: 1173231, 4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 1: 1091, 12: 823, 14: 365, 9: 20, 13: 12, 8: 6})
Class distribution after undersampling: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution Before SMOTE: Counter({4: 96794, 2: 71688, 10: 50789, 3: 5760, 7: 3322, 6: 3015, 5: 2928, 11: 1802, 12: 823, 14: 365, 0: 30, 9: 20, 13: 12, 1: 6, 8: 6})
Class Distribution After SMOTE: Counter({0: 96794, 1: 96794, 2: 96794, 3: 96794, 4: 96794, 5: 96794, 6: 96794, 7: 96794, 8: 96794, 9: 96794, 10: 96794, 11: 96794, 12: 96794, 13: 96794, 14: 96794})


In [18]:
# Train the fine-tuned model
history = fine_tuned_model.fit(
    X_train_resampled_30,  # Features from CICIDS2017
    y_train_resampled_30,  # Labels from CICIDS2017
    validation_data=(X_val_30, y_val_30),  # Validation set
    epochs=500,  # Adjust based on the dataset size
    batch_size=2000,  # Adjust batch size as needed
    verbose=2,
    callbacks=[model_checkpoint]
)

Epoch 1/500

Epoch 1: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.8038 - loss: 1414.1947 - val_accuracy: 0.4307 - val_loss: 66413.4453
Epoch 2/500

Epoch 2: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.8088 - loss: 1345.9900 - val_accuracy: 0.4105 - val_loss: 64947.0156
Epoch 3/500

Epoch 3: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.8118 - loss: 1302.4585 - val_accuracy: 0.4388 - val_loss: 63517.6094
Epoch 4/500

Epoch 4: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.8115 - loss: 1333.7645 - val_accuracy: 0.3790 - val_loss: 67022.0312
Epoch 5/500

Epoch 5: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.8131 - loss: 1257.5631 - val_accuracy: 0.4617 - val_loss: 59428.5469
Epoch 6/500

Epoch 6: val_accuracy did not improve from 0.64073
726/726 - 2s - 2ms/step - accuracy: 0.8129 - loss: 1252.4623 - val_accuracy: 0.41